In [ ]:
# 多次元時系列データの前処理ノートブック

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 0. 疑似データの生成
np.random.seed(0)
date_range = pd.date_range(start='2023-01-01', periods=100, freq='D')
data = {
    'timestamp': date_range,
    'dim1': np.random.normal(0, 1, size=len(date_range)),
    'dim2': np.random.normal(5, 2, size=len(date_range)),
    'dim3': np.random.normal(-3, 1.5, size=len(date_range))
}
df = pd.DataFrame(data)
df.set_index('timestamp', inplace=True)

def plot_in_grid(dataframes, titles, num_cols=3):
    num_rows = (len(dataframes) + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
    axes = axes.flatten()

    for i, (df, title) in enumerate(zip(dataframes, titles)):
        axes[i].plot(df)
        axes[i].set_title(title)
        axes[i].set_xlabel('Time')
        axes[i].set_ylabel('Value')
        if isinstance(df, pd.DataFrame):
            axes[i].legend(df.columns)
        else:
            axes[i].legend([df.name])

    for i in range(len(dataframes), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

# 1. 元データの時系列プロットの作成
plot_in_grid([df[col] for col in df.columns], [f'Original Time Domain Data - {col}' for col in df.columns])

# 2. 時間粒度を揃える
# リサンプリング (1日の間隔にリサンプリング)
resampled_df = df.resample('D').mean()

# 補完処理 (前方補完を使用)
filled_df = resampled_df.ffill()

# 元データと処理後のデータをプロットして比較
plot_in_grid(
    [pd.concat([df[col], filled_df[col]], axis=1, keys=['Original', 'Resampled and Filled']) for col in df.columns],
    [f'Comparison of Original and Resampled Data - {col}' for col in df.columns]
)

# 3. データの正規化
# 平均値±標準偏差でスケーリング (標準化)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(filled_df)
scaled_df = pd.DataFrame(scaled_data, index=filled_df.index, columns=filled_df.columns)

# 最小-最大値の幅で規格化 (正規化)
min_max_scaler = MinMaxScaler()
normalized_data = min_max_scaler.fit_transform(filled_df)
normalized_df = pd.DataFrame(normalized_data, index=filled_df.index, columns=filled_df.columns)

# 正規化データのプロット
plot_in_grid(
    [pd.concat([scaled_df[col], normalized_df[col]], axis=1, keys=['Standardized', 'Normalized']) for col in df.columns],
    [f'Comparison of Standardized and Normalized Data - {col}' for col in df.columns]
)

# 4. 時間スケールの変更 (平滑化)
# 移動平均 (7日の移動平均)
moving_avg_df = filled_df.rolling(window=7).mean()

# 加重移動平均 (WMA)
weights = np.arange(1, 8)
wma_df = pd.DataFrame(index=filled_df.index, columns=filled_df.columns)
for column in filled_df.columns:
    wma_df[column] = np.convolve(filled_df[column], weights/weights.sum(), mode='same')

# 単純指数平滑法 (SES)
ses_df = filled_df.ewm(alpha=0.2).mean()

# 平滑化データのプロット
plot_in_grid(
    [pd.concat([filled_df[col], moving_avg_df[col], wma_df[col], ses_df[col]], axis=1, 
               keys=['Original', 'Moving Average', 'Weighted Moving Average', 'Simple Exponential Smoothing']) 
     for col in df.columns],
    [f'Comparison of Smoothing Techniques - {col}' for col in df.columns]
)
